In [1]:
import os
os.environ["OMP_NUM_THREADS"] = "1"

import numpy as np
import pandas as pd
import sys,os
import random
import copy

import matplotlib.pyplot as plt
import seaborn as sns

from utils.eval import find_best_matches, generate_exprs

from methods import NMF, PCA, sparse_PCA, moCluster, MOFA2, iClusterPlus

from methods.utils import interpret_results, resultsHandler

from pathlib import Path
import multiprocessing as mp

from contextlib import redirect_stdout
# Settings for simulated data
N_GENES_LIST = ['5', '50', '500']
SCENARIOS = ['A', 'B', 'C']


In [ ]:
with open('test_methods_simulated_log.txt', 'w') as f:
    with redirect_stdout(f):
        
        for METHOD in [sparse_PCA, NMF]:  # NMF, MOFA2, moCluster, sparse_PCA, iClusterPlus
            for SCENARIO in SCENARIOS:
                for N_GENES in N_GENES_LIST:

                    #### prep file paths
                    # data files
                    data_path = f'/local/DESMOND2_data_simulated/simulated/{SCENARIO}'
                    FILE = f'{SCENARIO}.n_genes={N_GENES},m=4,std=1,overlap=yes.exprs_z.tsv'
                    file_path = os.path.join(data_path, FILE)
                    # output folder
                    output_path = os.path.join('/home/bba1401/data/unpast_simluated', METHOD.__name__.split('.')[1], SCENARIO, N_GENES)
                    # ground truth file
                    ground_truth_file = os.path.join(data_path, f'{SCENARIO}.n_genes={N_GENES},m=4,std=1,overlap=yes.biclusters.tsv')

                    if SCENARIO == 'A':
                        file_path = file_path.replace('overlap=yes', 'overlap=no')
                        ground_truth_file= ground_truth_file.replace('overlap=yes', 'overlap=no')

                    #### prep params
                    combinations = METHOD.generate_arg_list(file_path, output_path, ground_truth_file)
                    print(len(combinations))
                    #### run algorithm
                    with mp.Pool(20) as pool:
                        pool.map(METHOD.run_simulated, combinations)



/home/bba1401/anaconda3/envs/encore2/lib/python3.8/site-packages/sklearn/linear_model/_coordinate_descent.py:634: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 0.023825278219668178, tolerance: 0.0199
  model = cd_fast.enet_coordinate_descent_gram(


## Collect results

In [14]:
# Settings for simulated data
N_GENES_LIST = ['5', '50', '500']
SCENARIOS = ['A', 'B', 'C']

for METHOD in [NMF, sparse_PCA]:  # MOFA2, moCluster, NMF, sparse_PCA, iClusterPlus
    method_name = METHOD.__name__.split('.')[1]
    print(method_name)
    scores = []
    for SCENARIO in SCENARIOS:
        for N_GENES in N_GENES_LIST:
            
             #### prep file paths
            # data files
            data_path = f'/local/DESMOND2_data_simulated/simulated/{SCENARIO}'
            FILE = f'{SCENARIO}.n_genes={N_GENES},m=4,std=1,overlap=yes.exprs_z.tsv'
            file_path = os.path.join(data_path, FILE)
            # output folder
            output_path = os.path.join(f'/home/bba1401/data/unpast_simluated/{method_name}/{SCENARIO}/{N_GENES}')
            # ground truth file
            ground_truth_file = os.path.join(data_path, f'{SCENARIO}.n_genes={N_GENES},m=4,std=1,overlap=yes.biclusters.tsv')

            if SCENARIO == 'A':
                file_path = file_path.replace('overlap=yes', 'overlap=no')
                ground_truth_file= ground_truth_file.replace('overlap=yes', 'overlap=no')
            
            #### prep params
            combinations = METHOD.generate_arg_list(file_path, output_path, ground_truth_file)
            print(len(combinations), 'combinations')
            for i, comb in enumerate(combinations):
                if not i % 1000:
                    print(i)
                try:
                    performance, runtime = resultsHandler.evaluate_simulated(**comb)
                    scores.append({**performance, 'runtime': runtime, 'parameters': comb['output_path'], 'scenario': SCENARIO, 'gsize':N_GENES, 'run':comb['random_state'], 'seed':comb['random_state']})
                except Exception as e:
                    print(e)
                    continue
            
    pd.DataFrame(scores).to_csv(f'/home/bba1401/data/unpast_simluated/{method_name}/{method_name}_ABC.tsv', sep='\t')

NMF
14250 combinations
0



KeyboardInterrupt



In [10]:
import glob
import os

In [54]:
files = glob.glob('/home/bba1401/data/unpast_simluated/NMF/*/*/*/*/*/*/*/*/*/*/*/max_iter=200/*', recursive=True)

In [55]:
# files = glob.glob('/home/bba1401/data/unpast_real/METABRIC/NMF/*/*/*/*/*/*/*/*/*/*', recursive=True)

In [56]:
files

[]

In [7]:
comb = [x for x in combinations if x['k'] is None][0]

In [42]:
# defaults = 'max_iter=5000'

# for f in files:
#     splitted = f.split('/')
#     splitted = [*splitted[:-1], defaults, splitted[-1]]
#     dirpath = [*f.split('/')[:-1], defaults]
#     os.makedirs('/'.join(dirpath), exist_ok=True)
#     f_new = '/'.join(splitted)
#     os.rename(f, f_new)

In [52]:
f_new

'/home/bba1401/data/unpast_real/METABRIC/NMF/k=3/init=nndsvd/tol=0.0001/random_state=5/alpha_W=0.2/alpha_H=0.2/shuffle=False/solver=cd/beta_loss=frobenius/max_iter=5000/runtime.txt'

In [10]:
m = np.zeros([2, 5])

In [34]:
np.random.randint([5, 5, 5], 5)

ValueError: low >= high

In [11]:
import numpy as np
X = np.array([[1, 1], [2, 1], [3, 1.2], [4, 1], [5, 0.8], [6, 1]])
from sklearn.decomposition import NMF
model = NMF(n_components=2, init='random', random_state=0)
W = model.fit_transform(X)
H = model.components_

In [12]:
import json



In [16]:
pd.DataFrame(W)

,0,1
0,0.000000,0.468807
1,0.556995,0.389415
2,1.003316,0.419254
3,1.673400,0.229269
4,2.343493,0.039280
5,2.789815,0.069118


In [15]:
H

array([[2.09783018, 0.30560234],
       [2.13443044, 2.13171694]])